In [ ]:
!pip install easyocr

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr
     

In [ ]:
cars_cascade = cv2.CascadeClassifier('haarcascade_car.xml')

In [ ]:
def Simulator():
    CarVideo = cv2.VideoCapture('cars.mp4')
    count = 0
    while CarVideo.isOpened():
        ret, frame = CarVideo.read()
        if ret:        
            # cv2.imshow('frame', cars_frame)
            cars = cars_cascade.detectMultiScale(frame, 1.15, 4)

            for (x, y, w, h) in cars:
                cropped = cv2.rectangle(frame, (x+1, y+1), (x+w,y+h), color=(255, 0, 0), thickness=2)
                # cv2.imshow(cropped)
                silpa = frame[y:y+h, x:x+w]
                cv2.imwrite('frame%d.jpg' %count, silpa)
                count += 1
        else:
            break

    CarVideo.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    Simulator()


In [ ]:
#Reading the image
img = cv2.imread('frame23.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))#using another colour conversion since matplotlib expects rgb
     

In [ ]:
#finding edges for locating
bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
edged = cv2.Canny(bfilter, 30, 200) #Edge detection
plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))
     

In [ ]:
#finding contours
keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(keypoints)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

In [ ]:
location = None
for contour in contours:
    approx = cv2.approxPolyDP(contour, 10, True)
    if len(approx) == 4:
        location = approx
        break

In [ ]:
print(location)

In [ ]:
mask = np.zeros(gray.shape, np.uint8)
new_image = cv2.drawContours(mask, [location], 0,255, -1)
new_image = cv2.bitwise_and(img, img, mask=mask)
     

In [ ]:
plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))

In [ ]:
(x,y) = np.where(mask==255)
(x1, y1) = (np.min(x), np.min(y))
(x2, y2) = (np.max(x), np.max(y))
cropped_image = gray[x1:x2+1, y1:y2+1]

In [ ]:
plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

In [ ]:
#using ocr
reader = easyocr.Reader(['en'])
result = reader.readtext(cropped_image)
result

In [ ]:
#rendering result
text = result[0][-2]
font = cv2.FONT_HERSHEY_SIMPLEX
res = cv2.putText(img, text=text, org=(approx[0][0][0]-60, approx[1][0][1]+60), fontFace=font, fontScale=0.75, color=(0,255,0), thickness=2, lineType=cv2.LINE_AA)
res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3)
plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))